In [ ]:
!pip install openai langchain tiktoken chromadb
# https://python.langchain.com/docs/integrations/llms/llm_caching
# https://www.linkedin.com/pulse/how-cache-llm-calls-using-langchain-suman-mishra/
# importing all the dependencies
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.cache import InMemoryCache, SQLiteCache
import langchain
import chromadb
import tiktoken
import pandas as pd
import os
#langchain.llm_cache = InMemoryCache()
#langchain.llm_cache = SQLiteCache(database_path= "langchain.db")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml)

In [ ]:
##########################################################
#### IMPORT BOTH FILES BEFORE RUNNING THIS CELL ##########
##########################################################

# --- Step 1 ---
f = open("/content/api_key.txt")
api_key = f.read()
chat = ChatOpenAI(openai_api_key=api_key, temperature=0)

# --- Step 2 ---
df = pd.read_csv("/content/sdlms_dev.performance_reviews.csv")
len_entries = len(df)

#print("\n")
print(f"Total number of entries : {len_entries} unicorns")
print("=====================================")
df["name"]

Total number of entries : 10 unicorns


0                        Angad
1                 Gaurav Dahal
2                  Anaya Sutar
3                prajesheleven
4                   Gowthami 0
5    TARUGU UMA MAHESWAR REDDY
6                 Fardin Kamal
7                Samidha Kumar
8               Hinglaj Tanwar
9                  Artuha Paul
Name: name, dtype: object

In [ ]:
def weekly_report(x):

  # dependencies
  langchain.llm_cache = SQLiteCache(database_path= "langchain.db")

  question_list = [
  "How would you quantify your contributions (of ...",
  "What are the expected figures?",
  "What are the actual figures?",
  "Did you understand how your project lead quant...",
  "What bottlenecks are slowing you down?",
  "Why do you think you're facing this bottleneck?",
  "Would you like to create a process to solve th...",
  "What are your milestones for the next week?",
  "Why have you set these milestones?",
  "How are you improving the quality and quantity...",
  "How do you plan to achieve these milestones?",
  "How are you assessing your work?",
  "Do you rely on senior feedback or do you have ...",
  "If your project has a long gestation period an...",
  ]

  # setting up data points
  candidiate_fact_sheet = df.iloc[x]
  answers = candidiate_fact_sheet[3:-6]
  deliverables = candidiate_fact_sheet[4:6]
  name = candidiate_fact_sheet[1]
  questions_d = question_list[1:3]
  name = df['name'][x]
  team = df['departmentName'][x]

  # for bottlenecks
  template_string = """Based on the {name}, the intern's performance review \
  raise only 3-4 serious red-flags in bullet-points \
  using the question's list to the answers.\nQuestions:{question_list}\nAnswers:{answers} \
  Do not mentions questions in the summary.
  Raise atmost 4 red-flags.
  """
  prompt_template = ChatPromptTemplate.from_template(template_string)
  summary = prompt_template.format_messages(name = name,
                                            answers = answers,
                                            question_list = question_list
                                            )
  response = chat(summary)

  # for deliverables
  template_string_d = """Based on the {name}, the intern's performance review \
  raise only 3-4  bullet-points \
  using the question's list to the answers.\nQuestions:{question_list}\nAnswers:{answers} \
  Swap 'expected figures' with 'expected numbers' and 'actual figures' with 'actual nmbers'. \
  Do not mentions questions.
  """

  prompt_template_d = ChatPromptTemplate.from_template(template_string_d)
  summary_d = prompt_template.format_messages(name = name,
                                            answers = deliverables,
                                            question_list = questions_d
                                            )
  response_d = chat(summary_d)

  # SequentialChain
  template1 = "Give a summary of an intern's performance review\n{review}"
  prompt1 = ChatPromptTemplate.from_template(template1)
  chain1 = LLMChain(llm=chat, prompt = prompt1, output_key="review_summary")
  template2 = "Identify Key employee weakness in this review summary\n{review_summary}"
  prompt2 = ChatPromptTemplate.from_template(template2)
  chain2 = LLMChain(llm=chat, prompt = prompt2, output_key="weaknesses")
  template3 = "Create a personalized plan 'Personalized Plan to Address and Fix Key Employee Weaknesses:' to help address and fix these weaknesses with at most 4 pointers.\n{weaknesses}"
  prompt3 = ChatPromptTemplate.from_template(template3)
  chain3 = LLMChain(llm=chat, prompt = prompt3, output_key="final_plan")

  # Setting up SequentialChains
  seq_chain = SequentialChain(chains=[chain1, chain2, chain3],
                              input_variables=["review"],
                              output_variables=["review_summary", "weaknesses", "final_plan"],
                              verbose=False)
  results = seq_chain(candidiate_fact_sheet)
  print("WEEKLY REPORT:")
  print("________")
  print(f"Name = {name}\n")
  print(f"Team = {team}")
  print("\n")
  print("Summary:")
  print(results['review_summary'])
  print("\n")
  print("Deliverables:")
  print("______")
  print("NOTE: Here 'actual figures' represent 'actual numbers' and 'expected figures' represents 'expected numbers'.")
  print(response_d.content)
  print("\n")
  print("Bottlenecks:")
  print(response.content)
  print("\n")
  print("Suggestions")
  print(results["final_plan"])

In [ ]:
weekly_report(4)

WEEKLY REPORT:
---------------
Name = Gowthami 0

Team = Career Acceleration Lab


Summary:
The intern, Gowthami 0, has shown good performance in their role. They have successfully come up with 5 data points for the LPS index and have developed a method to quantify the index. They have faced difficulties in understanding the progress of the project, but this bottleneck is considered intrinsic. They have tried to go through all the relevant links and have focused on the DTthon courses to understand the team's objectives. They have been completing the most prioritized tasks and have been reviewing their progress with the team. They have been documenting the process and progress and have observable metrics to assess their performance. They are also documenting the reasonings behind long-gestation tasks. The performance review was conducted during the week of August 14th to August 20th, 2023.


Deliverables:
---------------
NOTE: Here 'actual figures' represent 'actual numbers' and 'expect